# Câu hỏi 4-5-6
---

## Import các thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr, spearmanr

## Load dữ liệu

In [ ]:
df = pd.read_csv('football_players_dataset.csv')
df.head()

---
# **CÂU HỎI 1: CẦU THỦ TỪ GIẢI ĐẤU NÀO CÓ XU HƯỚNG BỊ OVERVALUE HOẶC UNDERVALUE NHẤT?**

### 1. Lợi ích của việc trả lời câu hỏi

- Tối ưu chiến lược scouting: CLB có thể tập trung tìm kiếm cầu thủ từ các giải bị undervalue để mua với giá thấp hơn giá trị thực tế, tiết kiệm ngân sách chuyển nhượng.
- Quyết định bán cầu thủ: Nếu CLB đang thi đấu ở giải bị overvalue, có thể tận dụng để bán cầu thủ với giá cao hơn giá trị thực.
- Đánh giá rủi ro đầu tư: Nhà đầu tư và CLB tránh được việc mua cầu thủ từ các giải có "bong bóng giá", giảm thiểu rủi ro tài chính.

### 2. Mô tả thuật toán

1. Train prediction model: Sử dụng Random Forest để dự đoán market_value dựa trên performance metrics (không bao gồm league)
2. Tính residuals: Residual = Actual market_value - Predicted market_value
   - Residual > 0: Overvalued (giá thực tế cao hơn dự đoán)
   - Residual < 0: Undervalued (giá thực tế thấp hơn dự đoán)
3. Aggregate by league: Group residuals theo league, tính mean, median và visualize distribution
4. Statistical significance: Kiểm định xem sự khác biệt có ý nghĩa thống kê không

### 3. Tiền xử lý dữ liệu

In [ ]:
df_model = df[df['market_value'].notna()].copy()

feature_cols = [
    'age', 'height', 'appearances', 'minutes_played', 'minutes_per_game',
    'goals', 'assists', 'goals_per_90', 'assists_per_90', 
    'npxg_per90', 'xag_per90', 'shots_per_90', 'shots_on_target_per90',
    'key_passes_per90', 'passes_completed_per90', 'pass_completion_pct',
    'progressive_passes_per90', 'take_ons_per90', 'tackles_per_90',
    'interceptions_per90', 'blocks_per90', 'aerials_won_per90'
]

df_model = df_model[feature_cols + ['market_value', 'league']].copy()
df_model[feature_cols] = df_model[feature_cols].fillna(df_model[feature_cols].median())
if 'position' in df.columns:
    le_pos = LabelEncoder()
    df_model['position_encoded'] = le_pos.fit_transform(df['position'].fillna('Unknown'))
    feature_cols.append('position_encoded')

print(f"Dataset shape: {df_model.shape}")
print(f"Features used: {len(feature_cols)}")

In [ ]:
X = df_model[feature_cols]
y = df_model['market_value']
leagues = df_model['league']

X_train, X_test, y_train, y_test, league_train, league_test = train_test_split(
    X, y, leagues, test_size=0.2, random_state=42
)

print("Training Random Forest model...")
rf_model = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

train_score = rf_model.score(X_train, y_train)
test_score = rf_model.score(X_test, y_test)

print(f"Model R² score (train): {train_score:.3f}")
print(f"Model R² score (test): {test_score:.3f}")

In [ ]:
df_model['predicted_value'] = rf_model.predict(X)
df_model['residual'] = df_model['market_value'] - df_model['predicted_value']
df_model['residual_pct'] = (df_model['residual'] / df_model['predicted_value']) * 100

print("Data preprocessing completed!")
df_model[['market_value', 'predicted_value', 'residual', 'residual_pct', 'league']].head()

### 4. Trực quan hóa kết quả

In [ ]:
league_analysis = df_model.groupby('league').agg({
    'residual': ['mean', 'median', 'count'],
    'residual_pct': 'mean'
}).round(2)

league_analysis.columns = ['mean_residual', 'median_residual', 'count', 'mean_residual_pct']
league_analysis = league_analysis[league_analysis['count'] >= 10].sort_values('mean_residual')

print("League analysis summary:")
league_analysis.head(10)

In [ ]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

axes[0].barh(league_analysis.index, league_analysis['mean_residual'], 
             color=['red' if x < 0 else 'green' for x in league_analysis['mean_residual']])
axes[0].axvline(x=0, color='black', linestyle='--', linewidth=1)
axes[0].set_xlabel('Mean Residual (€)', fontsize=12)
axes[0].set_title('League Valuation Bias: Overvalued (Green) vs Undervalued (Red)', 
                  fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

axes[1].barh(league_analysis.index, league_analysis['mean_residual_pct'],
             color=['red' if x < 0 else 'green' for x in league_analysis['mean_residual_pct']])
axes[1].axvline(x=0, color='black', linestyle='--', linewidth=1)
axes[1].set_xlabel('Mean Residual (%)', fontsize=12)
axes[1].set_title('Percentage Deviation from Predicted Value by League', 
                  fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
top_leagues = league_analysis.nlargest(8, 'count').index

fig, ax = plt.subplots(figsize=(14, 8))
df_top = df_model[df_model['league'].isin(top_leagues)]

sns.boxplot(data=df_top, x='residual', y='league', palette='coolwarm', ax=ax)
ax.axvline(x=0, color='black', linestyle='--', linewidth=2, label='Perfect prediction')
ax.set_xlabel('Residual (Actual - Predicted market value in €)', fontsize=12)
ax.set_ylabel('League', fontsize=12)
ax.set_title('Distribution of market value residuals by league', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
most_overvalued = league_analysis.nlargest(3, 'mean_residual').index
most_undervalued = league_analysis.nsmallest(3, 'mean_residual').index
leagues_to_plot = list(most_overvalued) + list(most_undervalued)

fig, ax = plt.subplots(figsize=(12, 8))
for league in leagues_to_plot:
    df_league = df_model[df_model['league'] == league]
    ax.scatter(df_league['predicted_value'], df_league['market_value'], 
               label=f"{league} ({len(df_league)} players)", alpha=0.6, s=50)

max_val = max(df_model['predicted_value'].max(), df_model['market_value'].max())
ax.plot([0, max_val], [0, max_val], 'k--', linewidth=2, label='Perfect prediction')

ax.set_xlabel('Predicted Market Value (€)', fontsize=12)
ax.set_ylabel('Actual Market Value (€)', fontsize=12)
ax.set_title('Predicted vs Actual Market Value: Most over/undervalued leagues', 
             fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=9)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

### 5. Kết luận

In [ ]:
print("\nMost overvalued leagues:")
print(league_analysis.nlargest(5, 'mean_residual')[['mean_residual', 'mean_residual_pct', 'count']])
print("\nMost undervalued leagues:")
print(league_analysis.nsmallest(5, 'mean_residual')[['mean_residual', 'mean_residual_pct', 'count']])

**Nhận xét:**
- Các giải đấu bị overvalue thường là
- Các giải đấu bị undervalue thường là
- Nguyên nhân có thể do

---
# **CÂU HỎI 2: THỜI LƯỢNG THI ĐẤU CÓ LÀM TĂNG GIÁ TRỊ CỦA CẦU THỦ KHÔNG?**

### 1. Lợi ích của việc trả lời câu hỏi

- Chiến lược phát triển cầu thủ trẻ: CLB biết được việc cho cầu thủ trẻ ra sân nhiều có tác động tích cực đến giá trị hay không, từ đó quyết định chính sách cho mượn hoặc thi đấu.
- Đàm phán hợp đồng: Cầu thủ và agent có dữ liệu để chứng minh rằng thời gian thi đấu ảnh hưởng đến giá trị thị trường, hỗ trợ đàm phán lương và điều khoản.
- Đánh giá cầu thủ dự bị: Phát hiện liệu cầu thủ ít ra sân nhưng hiệu quả cao có bị undervalue hay không, tạo cơ hội tìm kiếm "hidden gems".

### 2. Mô tả thuật toán

1. Correlation analysis: Tính Pearson và Spearman correlation giữa playing time metrics với market_value
2. Partial correlation: Kiểm soát ảnh hưởng của performance metrics (goals, assists) để xem playing time có tác động độc lập không
3. Segmentation analysis: Chia thành nhóm theo minutes_played (low/medium/high) và so sánh market_value trung bình, kiểm soát age và position
4. Regression analysis: Thêm playing time vào regression model và kiểm tra coefficient significance

### 3. Tiền xử lý dữ liệu

In [ ]:
df_clean = df[df['market_value'].notna()].copy()

df_clean['total_minutes'] = df_clean['minutes_played'].fillna(0)
df_clean['apps'] = df_clean['appearances'].fillna(0)
df_clean['mins_per_game'] = df_clean['minutes_per_game'].fillna(0)

mask = (df_clean['mins_per_game'] == 0) & (df_clean['apps'] > 0)
df_clean.loc[mask, 'mins_per_game'] = df_clean.loc[mask, 'total_minutes'] / df_clean.loc[mask, 'apps']

print(f"Total players analyzed: {len(df_clean)}")

In [ ]:
df_clean['playing_time_category'] = pd.cut(
    df_clean['total_minutes'],
    bins=[0, 900, 2000, float('inf')],
    labels=['Low (<900 min)', 'Medium (900-2000 min)', 'High (>2000 min)']
)

print("\nPlaying time distribution:")
print(df_clean['playing_time_category'].value_counts())

In [ ]:
df_clean['goals_total'] = df_clean['goals'].fillna(0)
df_clean['assists_total'] = df_clean['assists'].fillna(0)

df_clean['efficiency_score'] = (
    (df_clean['goals_per_90'].fillna(0) * 3) +
    (df_clean['assists_per_90'].fillna(0) * 2) +
    (df_clean['key_passes_per90'].fillna(0) * 1)
) / 6

print("Efficiency score statistics:")
print(df_clean['efficiency_score'].describe())

In [ ]:
scaler = StandardScaler()
numeric_cols = ['total_minutes', 'apps', 'mins_per_game', 'market_value', 
                'age', 'goals_per_90', 'assists_per_90', 'efficiency_score']
df_scaled = df_clean.copy()
df_scaled[numeric_cols] = scaler.fit_transform(df_clean[numeric_cols].fillna(0))

print("Data preprocessing completed!")

### 4. Trực quan hóa kết quả

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")

fig, ax = plt.subplots(figsize=(10, 8))

playing_time_vars = ['total_minutes', 'apps', 'mins_per_game']
performance_vars = ['goals_per_90', 'assists_per_90', 'efficiency_score', 'market_value']
corr_vars = playing_time_vars + performance_vars

corr_matrix = df_clean[corr_vars].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(corr_matrix, annot=True, fmt='.3f', cmap='RdYlGn', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            mask=mask, ax=ax)
ax.set_title('Correlation: Playing Time vs Performance vs Market Value', 
             fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

In [ ]:
print("\nKey correlations with market value")
for var in playing_time_vars:
    corr, p_value = pearsonr(df_clean[var].fillna(0), df_clean['market_value'])
    print(f"{var}: r={corr:.3f}, p-value={p_value:.4f}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

playing_time_metrics = [
    ('total_minutes', 'Total Minutes Played', 'Minutes'),
    ('apps', 'Number of Appearances', 'Appearances'),
    ('mins_per_game', 'Minutes Per Game', 'Minutes/Game')
]

for idx, (col, title, xlabel) in enumerate(playing_time_metrics):
    ax = axes[idx]
    
    ax.scatter(df_clean[col], df_clean['market_value'], alpha=0.3, s=30)
    z = np.polyfit(df_clean[col].fillna(0), df_clean['market_value'], 2)
    p = np.poly1d(z)
    x_line = np.linspace(df_clean[col].min(), df_clean[col].max(), 100)
    ax.plot(x_line, p(x_line), "r-", linewidth=2, label='Polynomial fit')
    
    corr, _ = pearsonr(df_clean[col].fillna(0), df_clean['market_value'])
    
    ax.set_xlabel(xlabel, fontsize=11)
    ax.set_ylabel('Market Value (€)', fontsize=11)
    ax.set_title(f'{title}\n(Correlation: {corr:.3f})', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

sns.boxplot(data=df_clean, x='playing_time_category', y='market_value', ax=axes[0])
axes[0].set_xlabel('Playing Time Category', fontsize=12)
axes[0].set_ylabel('Market Value (€)', fontsize=12)
axes[0].set_title('Market Value by Playing Time Category', fontsize=13, fontweight='bold')
axes[0].tick_params(axis='x', rotation=15)

for i, category in enumerate(df_clean['playing_time_category'].cat.categories):
    mean_val = df_clean[df_clean['playing_time_category'] == category]['market_value'].mean()
    axes[0].text(i, mean_val, f'μ={mean_val:.2f}M', ha='center', va='bottom', fontweight='bold')

X_eff = df_clean[['efficiency_score']].fillna(0)
y = df_clean['market_value']
lr = LinearRegression()
lr.fit(X_eff, y)
df_clean['mv_residual'] = y - lr.predict(X_eff)

sns.boxplot(data=df_clean, x='playing_time_category', y='mv_residual', ax=axes[1])
axes[1].axhline(y=0, color='red', linestyle='--', linewidth=2, label='Expected value')
axes[1].set_xlabel('Playing time category', fontsize=12)
axes[1].set_ylabel('Market value residual (€)', fontsize=12)
axes[1].set_title('Market value residual (After controlling for efficiency)', 
                  fontsize=13, fontweight='bold')
axes[1].tick_params(axis='x', rotation=15)
axes[1].legend()

plt.tight_layout()
plt.show()

### 5. Kết luận

In [ ]:
print("\nMarket value by playing time category")
summary = df_clean.groupby('playing_time_category')['market_value'].agg(['count', 'mean', 'median', 'std'])
print(summary.round(2))

print("\nMarket value residual by playing time (Efficiency-Adjusted)")
summary_residual = df_clean.groupby('playing_time_category')['mv_residual'].agg(['mean', 'median'])
print(summary_residual.round(2))

**Nhận xét:**
- Mối tương quan giữa thời gian thi đấu và giá trị thị trường là...
- Sau khi kiểm soát hiệu suất (efficiency), thời gian thi đấu có ảnh hưởng độc lập...
- Khuyến nghị cho CLB và cầu thủ...

---
# **CÂU HỎI 3: KHẢ NĂNG DỨT ĐIỂM ẢNH HƯỞNG THẾ NÀO ĐẾN MARKET VALUE CỦA TIỀN ĐẠO?**

### 1. Lợi ích của việc trả lời câu hỏi

- **Đánh giá tiềm năng tiền đạo:** CLB có thể xác định được tiền đạo nào có khả năng dứt điểm tốt nhưng chưa được định giá đúng mức, tạo cơ hội mua "hidden gems".
- **Chiến lược phát triển cầu thủ:** Hiểu được các chỉ số dứt điểm nào (shots accuracy, xG conversion, shot placement) quan trọng nhất để tập trung cải thiện kỹ năng cầu thủ trẻ.
- **Quyết định đầu tư:** So sánh giữa tiền đạo "volume shooter" (nhiều cú sút nhưng tỷ lệ chuyển hóa thấp) và "clinical finisher" (ít cú sút nhưng hiệu quả cao) để đưa ra quyết định mua bán phù hợp với phong cách đội bóng.

### 2. Mô tả thuật toán

1. **Filter forwards:** Lọc ra các cầu thủ ở vị trí tiền đạo (FW hoặc position chứa 'FW')
2. **Feature engineering:** Tạo các chỉ số dứt điểm:
   - Conversion rate: goals / shots (tỷ lệ chuyển hóa)
   - xG overperformance: (goals - xG) / xG (dứt điểm tốt hơn kỳ vọng)
   - Shot efficiency: shots_on_target / shots (độ chính xác)
3. **Correlation & regression analysis:** Phân tích mối quan hệ giữa shooting metrics và market_value
4. **Segmentation:** Chia tiền đạo thành các nhóm (volume shooter, clinical finisher, balanced) và so sánh market value

### 3. Tiền xử lý dữ liệu

In [ ]:
df_forwards = df[df['market_value'].notna()].copy()

# Filter by position - check if 'FW' is in position string
if 'position' in df_forwards.columns:
    df_forwards = df_forwards[df_forwards['position'].str.contains('FW', na=False)].copy()
    print(f"Total forwards analyzed: {len(df_forwards)}")
else:
    print("Warning: 'position' column not found. Using all players.")

df_forwards['shots_total'] = df_forwards['shots_per_90'] * (df_forwards['minutes_played'] / 90)
df_forwards['shots_total'] = df_forwards['shots_total'].fillna(0)

df_forwards['conversion_rate'] = np.where(
    df_forwards['shots_total'] > 0,
    (df_forwards['goals'].fillna(0) / df_forwards['shots_total']) * 100,
    0
)

df_forwards['xg_total'] = df_forwards['xg_per90'].fillna(0) * (df_forwards['minutes_played'].fillna(0) / 90)
df_forwards['xg_overperformance'] = np.where(
    df_forwards['xg_total'] > 0,
    ((df_forwards['goals'].fillna(0) - df_forwards['xg_total']) / df_forwards['xg_total']) * 100,
    0
)

df_forwards['shot_accuracy'] = df_forwards['shots_on_target_pct'].fillna(0)

print("\nShooting metrics created:")
print(f"- Conversion rate (goals/shots %): {df_forwards['conversion_rate'].mean():.2f}%")
print(f"- xG overperformance (%): {df_forwards['xg_overperformance'].mean():.2f}%")
print(f"- Shot accuracy (%): {df_forwards['shot_accuracy'].mean():.2f}%")

In [ ]:
shots_median = df_forwards['shots_per_90'].median()
conversion_median = df_forwards['conversion_rate'].median()

def classify_forward(row):
    if row['shots_per_90'] > shots_median and row['conversion_rate'] < conversion_median:
        return 'Volume Shooter'
    elif row['shots_per_90'] <= shots_median and row['conversion_rate'] >= conversion_median:
        return 'Clinical Finisher'
    elif row['shots_per_90'] > shots_median and row['conversion_rate'] >= conversion_median:
        return 'Elite Striker'
    else:
        return 'Below Average'

df_forwards['forward_type'] = df_forwards.apply(classify_forward, axis=1)

print("\nForward type distribution:")
print(df_forwards['forward_type'].value_counts())
print("\nAverage market value by forward type:")
print(df_forwards.groupby('forward_type')['market_value'].mean().sort_values(ascending=False))

In [ ]:
df_forwards_filtered = df_forwards[
    (df_forwards['appearances'].fillna(0) >= 5) & 
    (df_forwards['minutes_played'].fillna(0) >= 450)
].copy()

print(f"\nForwards with sufficient playing time: {len(df_forwards_filtered)}")
print(f"Removed {len(df_forwards) - len(df_forwards_filtered)} forwards with insufficient data")

### 4. Trực quan hóa kết quả

In [ ]:
shooting_metrics = ['shots_per_90', 'shots_on_target_per90', 'shot_accuracy', 
                    'conversion_rate', 'xg_per90', 'xg_overperformance', 'goals_per_90']

correlations = {}
for metric in shooting_metrics:
    if metric in df_forwards_filtered.columns:
        corr, p_val = pearsonr(
            df_forwards_filtered[metric].fillna(0), 
            df_forwards_filtered['market_value']
        )
        correlations[metric] = {'correlation': corr, 'p_value': p_val}

corr_df = pd.DataFrame(correlations).T.sort_values('correlation', ascending=False)
print("\n=== CORRELATIONS WITH MARKET VALUE ===")
print(corr_df.round(4))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['green' if x > 0 else 'red' for x in corr_df['correlation']]
ax.barh(corr_df.index, corr_df['correlation'], color=colors, alpha=0.7)
ax.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax.set_xlabel('Correlation with Market Value', fontsize=12)
ax.set_title('Shooting Metrics Correlation with Market Value (Forwards Only)', 
             fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

for idx, (metric, row) in enumerate(corr_df.iterrows()):
    ax.text(row['correlation'], idx, f"  {row['correlation']:.3f}", 
            va='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

axes[0].scatter(df_forwards_filtered['conversion_rate'], 
                df_forwards_filtered['market_value'], 
                alpha=0.5, s=50)

z = np.polyfit(df_forwards_filtered['conversion_rate'].fillna(0), 
               df_forwards_filtered['market_value'], 1)
p = np.poly1d(z)
x_line = np.linspace(df_forwards_filtered['conversion_rate'].min(), 
                     df_forwards_filtered['conversion_rate'].max(), 100)
axes[0].plot(x_line, p(x_line), "r-", linewidth=2, label='Linear fit')

corr, _ = pearsonr(df_forwards_filtered['conversion_rate'].fillna(0), 
                   df_forwards_filtered['market_value'])
axes[0].set_xlabel('Conversion Rate (%)', fontsize=12)
axes[0].set_ylabel('Market Value (€)', fontsize=12)
axes[0].set_title(f'Conversion Rate vs Market Value\n(Correlation: {corr:.3f})', 
                  fontsize=13, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

axes[1].scatter(df_forwards_filtered['xg_overperformance'], 
                df_forwards_filtered['market_value'], 
                alpha=0.5, s=50, color='orange')

z2 = np.polyfit(df_forwards_filtered['xg_overperformance'].fillna(0), 
                df_forwards_filtered['market_value'], 1)
p2 = np.poly1d(z2)
x_line2 = np.linspace(df_forwards_filtered['xg_overperformance'].min(), 
                      df_forwards_filtered['xg_overperformance'].max(), 100)
axes[1].plot(x_line2, p2(x_line2), "r-", linewidth=2, label='Linear fit')

corr2, _ = pearsonr(df_forwards_filtered['xg_overperformance'].fillna(0), 
                    df_forwards_filtered['market_value'])
axes[1].axvline(x=0, color='gray', linestyle='--', alpha=0.5, label='Expected performance')
axes[1].set_xlabel('xG Overperformance (%)', fontsize=12)
axes[1].set_ylabel('Market Value (€)', fontsize=12)
axes[1].set_title(f'xG Overperformance vs Market Value\n(Correlation: {corr2:.3f})', 
                  fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

forward_type_order = ['Elite Striker', 'Clinical Finisher', 'Volume Shooter', 'Below Average']
df_plot = df_forwards_filtered[df_forwards_filtered['forward_type'].isin(forward_type_order)]

sns.boxplot(data=df_plot, x='forward_type', y='market_value', 
            order=forward_type_order, palette='Set2', ax=axes[0])
axes[0].set_xlabel('Forward Type', fontsize=12)
axes[0].set_ylabel('Market Value (€)', fontsize=12)
axes[0].set_title('Market Value Distribution by Forward Type', fontsize=13, fontweight='bold')
axes[0].tick_params(axis='x', rotation=15)

for i, fwd_type in enumerate(forward_type_order):
    if fwd_type in df_plot['forward_type'].values:
        mean_val = df_plot[df_plot['forward_type'] == fwd_type]['market_value'].mean()
        axes[0].text(i, mean_val, f'μ={mean_val:.2f}M', 
                     ha='center', va='bottom', fontweight='bold', fontsize=9)

scatter = axes[1].scatter(df_forwards_filtered['shots_per_90'], 
                          df_forwards_filtered['conversion_rate'],
                          c=df_forwards_filtered['market_value'], 
                          cmap='viridis', s=60, alpha=0.6)

axes[1].axvline(x=shots_median, color='red', linestyle='--', alpha=0.5, label=f'Median shots ({shots_median:.2f})')
axes[1].axhline(y=conversion_median, color='blue', linestyle='--', alpha=0.5, label=f'Median conversion ({conversion_median:.2f}%)')

axes[1].text(shots_median * 1.5, conversion_median * 1.5, 'Elite Striker', 
             fontsize=10, ha='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
axes[1].text(shots_median * 0.5, conversion_median * 1.5, 'Clinical Finisher', 
             fontsize=10, ha='center', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
axes[1].text(shots_median * 1.5, conversion_median * 0.5, 'Volume Shooter', 
             fontsize=10, ha='center', bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))

plt.colorbar(scatter, ax=axes[1], label='Market Value (€)')
axes[1].set_xlabel('Shots per 90 minutes', fontsize=12)
axes[1].set_ylabel('Conversion Rate (%)', fontsize=12)
axes[1].set_title('Forward Classification: Shots Volume vs Conversion Rate', 
                  fontsize=13, fontweight='bold')
axes[1].legend(loc='upper right', fontsize=9)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

### 5. Kết luận

In [ ]:
print("\nForward type analysis summary:")
summary = df_forwards_filtered.groupby('forward_type').agg({
    'market_value': ['count', 'mean', 'median', 'std'],
    'shots_per_90': 'mean',
    'conversion_rate': 'mean',
    'goals_per_90': 'mean'
}).round(2)
print(summary)

print("\nTop 10 most valuable forwards:")
top_forwards = df_forwards_filtered.nlargest(10, 'market_value')[[
    'player_name', 'current_club', 'market_value', 'forward_type', 
    'goals_per_90', 'shots_per_90', 'conversion_rate', 'xg_overperformance'
]]
print(top_forwards.to_string(index=False))

print("\n=== UNDERVALUED CLINICAL FINISHERS (High conversion, low market value) ===")
clinical = df_forwards_filtered[
    (df_forwards_filtered['forward_type'] == 'Clinical Finisher') &
    (df_forwards_filtered['market_value'] < df_forwards_filtered['market_value'].quantile(0.5))
].nlargest(5, 'conversion_rate')[[
    'player_name', 'current_club', 'market_value', 'conversion_rate', 'goals_per_90', 'shots_per_90'
]]
print(clinical.to_string(index=False))

**Nhận xét:**
- Chỉ số dứt điểm có correlation mạnh nhất với market value là...
- So sánh giữa "Volume Shooter" và "Clinical Finisher": loại nào được định giá cao hơn?
- xG overperformance có vai trò như thế nào trong việc định giá tiền đạo?
- Khuyến nghị cho CLB khi tuyển mộ tiền đạo